# Modelo Random Forest

In [ ]:
# Importando as dependências necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Carregando arquivo com os resultados e falhas
df = pd.read_csv("tabela_modelo.csv")

In [ ]:
# Removendo a primeira coluna vazia
df.drop(df.columns[0], axis=1, inplace=True)
df

In [ ]:
columns = ['ID1 S OK', 'ID2 S OK', 'ID718 S OK', 'ID1 S NOK', 'ID2 S NOK', 'ID718 S NOK']

# Substituindo NaN por 0 nas colunas especificadas
df[columns] = df[columns].fillna(0)

In [ ]:
# Transformando datetime para numérico

df['ID1 temp med'] = pd.to_numeric(df['ID1 temp med'], errors='coerce')
df['ID2 temp med'] = pd.to_numeric(df['ID2 temp med'], errors='coerce')
df['ID718 temp med'] = pd.to_numeric(df['ID718 temp med'], errors='coerce')

In [ ]:
df.info()

In [ ]:
# Criar variáveis alvo e preditoras
df['TOTAL OK'] = df['ID1 S OK'] + df['ID2 S OK'] + df['ID718 S OK']
df['TOTAL NOK'] = df['ID1 S NOK'] + df['ID2 S NOK'] + df['ID718 S NOK']

# Definir a coluna 'RESULTADO' onde 1 = "OK" e 0 = "NOK" se houver ao menos um NOK
df['RESULTADO'] = (df['TOTAL NOK'] == 0).astype(int)

# Preparar os dados para a predição da falha
# Criar a coluna 'FALHA_OPER' indicando onde a falha pode ocorrer
df['FALHA_OPER'] = df[['ID1 S NOK', 'ID2 S NOK', 'ID718 S NOK']].idxmax(axis=1)
df['FALHA_OPER'] = df['FALHA_OPER'].map({'ID1 S NOK': 'ID1', 'ID2 S NOK': 'ID2', 'ID718 S NOK': 'ID718'})

# Se não houver nenhuma falha, definir 'FALHA_OPER' como 'Nenhuma'
df.loc[df['TOTAL NOK'] == 0, 'FALHA_OPER'] = 'Nenhuma'

# Features (X) e Target (y) para a predição de falha
X = df[['ID1 QT','ID1 temp med',
                'ID2 QT', 'ID2 temp med',
                'ID718 QT', 'ID718 temp med']]
y = df[['FALHA_OPER', 'ID1 S OK', 'ID1 S NOK', 'ID2 S OK', 'ID2 S NOK','ID718 S OK', 'ID718 S NOK']]

# Transformar a variável alvo (y) em numérica para o modelo de classificação
y_encoded = y['FALHA_OPER'].map({'ID1': 0, 'ID2': 1, 'ID718': 2, 'Nenhuma': 3})

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Treinar um modelo de Random Forest para prever a falha
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Prever os resultados no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar a performance do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['ID1', 'ID2', 'ID718', 'Nenhuma']))

# Adicionar a coluna 'FALHA_OPER' no DataFrame original com as previsões
df['FALHA_OPER'] = model.predict(X)
df['FALHA_OPER'] = df['FALHA_OPER'].map({1: 'ID1', 2: 'ID2', 718: 'ID718', 0: 'Nenhuma'})

# Exibir o DataFrame resumo com as novas colunas 'RESULTADO' e 'FALHA_OPER'
df

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
report = classification_report(y_test, y_pred, target_names=['ID1', 'ID2', 'ID718', 'Nenhuma'], output_dict=True)

report_df = pd.DataFrame(report).transpose()

accuracy = accuracy_score(y_test, y_pred)

print("Classification Report:\n")
print(report_df)
print(f"\nAcurácia: {accuracy:.2f}")


In [ ]:
# Calcular a matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2, 3])

# Criar um DataFrame para a matriz de confusão com rótulos
cm_df = pd.DataFrame(cm, index=['ID1', 'ID2', 'ID718', 'Nenhuma'], columns=['ID1', 'ID2', 'ID718', 'Nenhuma'])

# Plotar a matriz de confusão
plt.figure(figsize=(10, 7))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=False, 
            annot_kws={"size": 12}, linewidths=.5, linecolor='black')
plt.title('Matriz de Confusão')
plt.xlabel('Previsões')
plt.ylabel('Valores Reais')
plt.show()


In [ ]:
# Criar a matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2, 3])

# Criar a visualização
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['ID1', 'ID2', 'ID718', 'Nenhuma'])

# Plotar a matriz de confusão
fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(ax=ax, cmap=plt.cm.Blues, colorbar=True)

# Adicionar títulos e labels
plt.title('Matriz de Confusão do Modelo Random Forest')
plt.show()